In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
features = ['Name','Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Cabin', 'Embarked', 'Fare']
train_data[features].head()

def Cabin_correct(Cabin):
    if type(Cabin) != float:
        Cabin = Cabin[0]
    else:
        Cabin = 'None'
    return Cabin
    
train_data.Cabin = train_data.Cabin.apply(Cabin_correct)
test_data.Cabin = test_data.Cabin.apply(Cabin_correct)

train_data.Embarked.fillna(value = train_data.Embarked.mode()[0], inplace = True)

test_data.Fare.fillna(value = test_data.Fare.mean(), inplace = True)


def title_extract(Name):
    return Name[Name.find(',')+2:Name.find('.')]

train_data.Name = train_data.Name.apply(title_extract)
test_data.Name = test_data.Name.apply(title_extract)

train_data.Age.fillna(value = 0, inplace = True)
test_data.Age.fillna(value = 0, inplace = True)

mean_age = train_data.groupby('Name').mean()['Age']

for i in range(891):
    if(train_data['Age'][i] == 0):
        train_data['Age'][i] = mean_age[train_data['Name'][i]]
        
mean_age = test_data.groupby('Name').mean()['Age']

for i in range(418):
    if(test_data['Age'][i] == 0):
        test_data['Age'][i] = mean_age[test_data['Name'][i]]

scaler = StandardScaler()
train_data[['Age','Fare']] = scaler.fit_transform(train_data[['Age','Fare']])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
features = ['Name','Pclass', 'Sex', 'Age','Fare', 'Cabin', 'Embarked']

In [4]:
learning_rates = np.linspace(1,2,10)
for rate in learning_rates:
    catagorical_transformer = OneHotEncoder(handle_unknown = 'ignore')
    model = XGBClassifier(random_state = 42, use_label_encoder = False,
                         learning_rate = 0.03,
                         n_estimators = 150,
                         subsample = 0.5,
                          colsample_bytree = 1,
                          max_depth = 3,
                         eval_metric = 'error',
                         booster = 'gbtree',
                         gamma = 0.1,
                         min_child_weight = 1,
                         reg_alpha = 0.4)
                         
    
    pipeline = Pipeline(steps = [('preprocessor',catagorical_transformer),
                                ('model', model)
                                ])

    scores = cross_val_score(pipeline,
                             train_data[features],
                             train_data.Survived,
                            cv = 10,
                            scoring = 'accuracy')
    print('Accuracy =', scores.mean(),'learning_rate = ',rate)

Accuracy = 0.8350062421972535 learning_rate =  1.0
Accuracy = 0.8350062421972535 learning_rate =  1.1111111111111112
Accuracy = 0.8350062421972535 learning_rate =  1.2222222222222223
Accuracy = 0.8350062421972535 learning_rate =  1.3333333333333333
Accuracy = 0.8350062421972535 learning_rate =  1.4444444444444444
Accuracy = 0.8350062421972535 learning_rate =  1.5555555555555556
Accuracy = 0.8350062421972535 learning_rate =  1.6666666666666665
Accuracy = 0.8350062421972535 learning_rate =  1.7777777777777777
Accuracy = 0.8350062421972535 learning_rate =  1.8888888888888888
Accuracy = 0.8350062421972535 learning_rate =  2.0


In [5]:
help(XGBClassifier)
gamma = 0.1
max_depth = 3
reg_alpha = 0.5

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective='binary:logistic', use_label_encoder=True, **kwargs)
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |      use_label_encoder : bool
 |          (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new code,
 |          we recommend that you set this parameter to False.
 |  
 |      max_depth : int
 |          Maximum tree depth for base learners.
 |      learning_rate : float
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : int
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |      objective : string or callable
 |          Specify the learning task and the corresponding learning objective or
 |          a custom objective func

In [6]:
model = XGBClassifier(random_state = 42, use_label_encoder = False,
                         learning_rate = 0.03,
                         n_estimators = 150,
                         subsample = 0.5,
                          colsample_bytree = 1,
                          max_depth = 3,
                         eval_metric = 'error',
                         booster = 'gbtree',
                         gamma = 0.1,
                         min_child_weight = 1,
                         reg_alpha = 0.4)
pipeline = Pipeline(steps = [('preprocessor', OneHotEncoder(handle_unknown = 'ignore')),
                            ('model', model)])
pipeline.fit(train_data[features], train_data.Survived)
predictions = pipeline.predict(test_data[features])

In [7]:
output = pd.DataFrame({'PassengerId':test_data.PassengerId, 
                      'survived': predictions})
output.to_csv('XGBClassifier.csv',index = False)